# Generation of explanation
This notebook is created for limit the use of server GPU just for the creation of the explanation
 and not for run the all code

In [7]:
%pip install pandas
%pip install transformers
%pip install tqdm
%pip install torch
%pip install python-dotenv
%pip install bitsandbytes
%pip install accelerate
%pip install setuptools

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
  Using cached setuptools-70.0.0-py3-none-any.whl.metadata (5.9 kB)
Using cached setuptools-70.0.0-py3-none-any.whl (863 kB)
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pandas as pd
from dotenv import load_dotenv
from transformers import BitsAndBytesConfig, AutoTokenizer, AutoModelForCausalLM
from tqdm import tqdm
import os
import torch

In [ ]:
# this is the script for create the dataset with the use of the prompt already created
os.environ['TOKENIZERS_PARALLELISM'] = 'false'
device = 'cpu'
if torch.backends.mps.is_available():
    device = 'mps'
elif torch.cuda.is_available():
    device = 'cuda'

load_dotenv()

token = os.getenv('AUTH_TOKEN')

df_explainability = pd.read_csv('explainability/prompt_llm.csv')

model_id = 'meta-llama/Meta-Llama-3-8B-Instruct'
output_length = 1024

# use quantization to lower GPU usage
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, 
    bnb_4bit_use_double_quant=True, 
    bnb_4bit_quant_type="nf4", 
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id, token=token)
llm_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    quantization_config=bnb_config,
    token=token
)
print('Tokenizer and model loaded.')
explaination = pd.DataFrame(columns=['subject_id','session_id','explanation'])

for sub_id, session_id, prompt in tqdm(df_explainability.to_numpy()):
    inputs = tokenizer.apply_chat_template(
        [{'role': 'user', 'content': prompt}],
        add_generation_prompt = True,
        return_tensors = 'pt'
    )

    outputs = llm_model.generate(
        inputs,
        max_new_tokens = output_length,
        eos_token_id = [
          tokenizer.eos_token_id,
          tokenizer.convert_tokens_to_ids('<|eot_id|>')
        ],
        pad_token_id = tokenizer.eos_token_id,
        do_sample = True,
        temperature = 0.6,
        top_p = 0.9,
    )

    readable_output = tokenizer.decode(outputs[0][inputs.shape[-1]:], skip_special_tokens=True)
    explaination.loc[len(explaination)] = {
        'subject_id': sub_id, 
        'session_id': session_id,
        'explanation': readable_output
    }

explaination.to_csv(f'explainability/explanation.csv', encoding='utf-8')
